# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
DATA_FOLDER = "/home/vinz/Desktop/ADA/ADA2017-Tutorials/02 - Intro to Pandas/Data" # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

### Strategy

We will import and wrangle the data for each country separately and then combine everything into a single data frame.

As the data files already contain variables with nation wide statistics, we will use these values instead of manually aggregating the data city by city.

As specified on Mattermost, we track for both new cases and deaths the probable, suspected and confirmed values.

In [ ]:
def ebola_csv_iter(folder):
    """Utility function returning an iterator over the csv files in given folder"""
    path = ''.join([DATA_FOLDER, '/ebola/', folder, '/'])
    for file in os.listdir(path):
        if file.endswith(".csv"):
            yield file, ''.join([path, file])

In [ ]:
def associate_to_country(country, df):
    """Adds a country super index to the provided dataframe"""
    country = pd.DataFrame({
        'Country': np.repeat(country, len(df)),
        'Date': df.index
    })
    return country.merge(df, right_index=True, left_on='Date').set_index(['Country', 'Date'])

In [ ]:
def parse_num(x):
    try:
        return np.float32(x)
    except ValueError:
        return np.nan

### Guinea

In [ ]:
dfs = {}
tables_with_duplicates = []
interesting_fields = {
    #'New deaths registered today (confirmed)': 'Death confirmed',
    #'New deaths registered today (probables)': 'Death probable',
    #'New deaths registered today (suspects)': 'Death suspected',
    'Total deaths of suspects': 'Death suspected',
    'Total deaths of probables': 'Death probable',
    'Total deaths of confirmed': 'Death confirmed',
    'New cases of confirmed': 'New cases confirmed',
    'New cases of probables': 'New cases probable', 
    'New cases of suspects': 'New cases suspected'
}
for file, path in ebola_csv_iter('guinea_data'):
    df = pd.read_csv(path,
                     parse_dates=['Date'],
                     usecols=['Date', 'Description', 'Totals'],
                     converters={'Totals': parse_num})
    df.rename(columns={'Description': 'Variable', 'Totals': 'National'}, inplace=True)
    # Check for duplicates
    df = df[df.Variable.isin(interesting_fields)]
    if len(df.Variable.unique()) != len(df.Variable):
        tables_with_duplicates.append(file)
    dfs[file] = df

In [ ]:
tables_with_duplicates

As there are no duplicates, we can pivot all the dataframes to have a timeseries dataframe:

In [ ]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [ ]:
guinea = pd.concat(dfs).sort_index() # Create the data series: index=date
guinea = associate_to_country('Guinea', guinea).rename(columns=interesting_fields)

We started by using the fields:

 - `New deaths registered today (confirmed)`
 - `New deaths registered today (probables)`
 - `New deaths registered today (suspects)`
 
but they were empty for all days except one. Therefore we decided to use the cumulated values and perform the subtraction to obtain daily counts:

In [ ]:
deaths = guinea[['Death confirmed', 'Death probable', 'Death suspected']]
cleaned = deaths - deaths.shift(1) # row_i - row_{i - 1}
guinea.loc[:, ['Death confirmed', 'Death probable', 'Death suspected']] = cleaned

This operation relies on a very important assumption. 
We repeat the same computation on the data concerning Liberia and Sierra Leone.
We assume that, given that the counts are cumulative, if we are missing a value for day $j$ and we have a value for day $i > j$, the value for day $j$ is counted in day $i$. **In other words, even if data for some days is missing, the totals are correct.**

In [ ]:
guinea.head(n=10)

### Liberia

In [ ]:
dfs = {}
tables_with_duplicates = []
interesting_fields = {
    'Total death/s in confirmed cases': 'Death confirmed',
    'Total death/s in probable cases': 'Death probable',
    'Total death/s in suspected cases': 'Death suspected',
    'New case/s (confirmed)': 'New cases confirmed',
    'New Case/s (Probable)': 'New cases probable',
    'New Case/s (Suspected)': 'New cases suspected'
}
for file, path in ebola_csv_iter('liberia_data'):
    df = pd.read_csv(path,
                     parse_dates=['Date'],
                     usecols=['Date', 'Variable', 'National'],
                     converters={'National': parse_num})
    df = df[df.Variable.isin(interesting_fields)]
    # Check for duplicates
    if len(df.Variable.unique()) != len(df.Variable):
        tables_with_duplicates.append(file)
    dfs[file] = df

We remark that only a data file contains duplicate variables for the same day. Before dropping the duplicates, we look at the duplicated lines:

In [ ]:
tables_with_duplicates

In [ ]:
with_duplicates = dfs[tables_with_duplicates[0]]
with_duplicates[with_duplicates.duplicated(keep=False, subset='Variable')]

7 Variables are repeated twice.

In [ ]:
first = with_duplicates[with_duplicates.duplicated(keep='last', subset='Variable')][['Variable', 'National']]
second = with_duplicates[with_duplicates.duplicated(keep='first', subset='Variable')][['Variable', 'National']]
first = first.set_index('Variable')
second = second.set_index('Variable')

In [ ]:
second - first

We decided to keep the bigger values as the difference is not very relevant:

In [ ]:
with_duplicates.drop_duplicates(subset='Variable', keep='last', inplace=True)

In [ ]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [ ]:
liberia = pd.concat(dfs).sort_index()
liberia = associate_to_country('Liberia', liberia).rename(columns=interesting_fields)

In [ ]:
liberia.head(n=10)

However in this dataframe, the three deaths columns are *cumulative*. We can therefore subtract the value from the previous line.

In [ ]:
deaths = liberia[['Death confirmed', 'Death probable', 'Death suspected']]
cleaned = deaths - deaths.shift(1) # row_i - row_{i - 1}
liberia.loc[:, ['Death confirmed', 'Death probable', 'Death suspected']] = cleaned

In [ ]:
to_scale = liberia[-6:][['New cases probable', 'New cases suspected', 'New cases confirmed']]

In [ ]:
liberia.loc[-6:, ['New cases probable', 'New cases suspected', 'New cases confirmed']] = to_scale - to_scale.shift(1)

### Sierra Leone

In [ ]:
dfs = {}
tables_with_duplicates = []
interesting_fields = {
    'death_confirmed': 'Death confirmed',
    'death_probable': 'Death probable',
    'death_suspected': 'Death suspected',
    'new_confirmed': 'New cases confirmed',
    'new_probable': 'New cases probable',
    'new_suspected': 'New cases suspected'
}

for file, path in ebola_csv_iter('sl_data'):
    df = pd.read_csv(path,
                     parse_dates=['date'],
                     usecols=['date', 'variable', 'National'], 
                     converters={'National': parse_num})
    df.rename(columns={'date': 'Date', 'variable': 'Variable'}, inplace=True)
    df = df[df.Variable.isin(interesting_fields)]
    if len(df.Variable.unique()) != len(df.Variable):
        tables_with_duplicates.append(file)
    dfs[file] = df

In [ ]:
tables_with_duplicates

In [ ]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [ ]:
sl = pd.concat(dfs).sort_index()
sl = associate_to_country('Sierra Leone', sl).rename(columns=interesting_fields)

In [ ]:
sl.head(n=10)

The fields concerning the number of deaths are cumulative. We apply the same treatment as in the previous cases:

In [ ]:
deaths = sl[['Death confirmed', 'Death probable', 'Death suspected']]
cleaned = deaths - deaths.shift(1) # row_i - row_{i - 1}
sl.loc[:, ['Death confirmed', 'Death probable', 'Death suspected']] = cleaned

In [ ]:
del dfs
del tables_with_duplicates

### Combining the data

In [ ]:
# Concatenate all dataframes
df = pd.concat([guinea, liberia, sl])

### Computing means

In [ ]:
def month_average(group):
    # Only divide by the number of days counted in the set.
    registered_days = group.index.get_level_values(1).max().day
    return group.sum() / registered_days

In [ ]:
import calendar
# Compute mean per month per country. 
results = df.groupby(lambda row: (row[0], row[1].month))\
            .agg(month_average)

# Give computation result a nice index
results.index = pd.MultiIndex.from_tuples(results.index, names=['Country', 'Month'])\
                  .map(lambda i: (i[0], calendar.month_name[i[1]]))

In [ ]:
results

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Setup for location of the dataset of the task 2
MICROBIOME_FOLDER = DATA_FOLDER + "/microbiome"


### Basic analysis of the file formatting:

***
**For the files MIDn.xls with n in [1,9]**

**column 1:**

We see that the first column in the files contain the scientific classification of the microbiomes
Altough it could be kept as a single string, it would have more meaning if splitted
The scientific classification contains the following subdivisions (https://en.wikipedia.org/wiki/Taxonomic_rank): 

       Domain, Kingdom, Phylum, Class, Order, Family, Genus, Species
       
 With this classification, we have a problem: only 6 strings are given in the data set, whilst we have 8 potential divisions in the classification.
 
 First, we note that in the scientific classification, Kingdom is not used with Bacterias (https://en.wikipedia.org/wiki/Bacteria) and in Archeas, it is always the same as the Phylum (https://en.wikipedia.org/wiki/Archaea). After checking the strings in the data set, it indeed seems that the Kingdom is never given. Therefore, **we will not use Kingdom as a division**.
 
 Also, Species only has a sense in the Eucaryote Domain, for which we have no data, so **we will not use Species as a division**.
 
 In addition, after playing with the data, we found cases where the Family can be named: "Incertae Sedis", which would be classified as the family and genus in our classification. Therefore, we need to check for those cases and re-concatenate the strings to return a proper list in every cases.

In [ ]:
# Lets keep those names in a list for further use
scientific_classification = ["domain", "phylum", "class", "order", "family", "genus"]

**Column 2:**

We see that the second column contains an integer value. We can suppose that this value is the number of samples
containing the genus defined by the first column. There's no title to the column, so we don't know yet where
those values came from.

***

**For the file "Metadata.xls"**

**Column 1:**

Titled **"BARCODE"**, give xls file identifier for which the two other columns give more information

**Column 2:**

Titled **"GROUP"**, gives the groups from which each dataset has been sampled. Two informations are contained in those groups, the first is given by "NEC", "Control" or "EXTRACTION CONTROL" and the second is the numbering of the group (which is likely a phase of test), either "1" or "2". We will split those informations in two columns, because while using the dataset, we might want to combine all the "NEC" patients or all the patient for a specific phase.

**Column 3:**

Titled **"SAMPLE"**, gives the type of sample that was taken, either tissue, stool or NA. Each group had both types of samples taken. 

***

### Desired formating of the data after analysis

The simple analysis above tells us what are the columns that we will want in our Data Frame

1. **6** column for classification, 1 for each classifier in the "scientific_classification" list. This will caracterise each microbiome individually.

2. **1** column will contain the value associated with each microbiome measurement.

3. **2** columns describing the group of the sample from the metadata. The first column will be called **Group Type** and will contain the "NEC", "Control" or "EXTRACTION CONTROL" value. The second column will be called **Group Phase** and will contain either "1", "2" or "unknow", "unknow" will be used for the "EXCTRACTON CONTROL" group.

4. **1** column describing the type of sample taken from the metadata. This column will be called **Sample** This will be either "tissue", "stool" or "unknow", "unknow" in the case of the EXTRACTION CONTROL group.

For a total of **10** columns


In [ ]:
metadata_list = ["group_type", "group_phase", "sample"]

In [ ]:
df_col_list = scientific_classification + metadata_list + ["value"]

In [ ]:
print(len(df_col_list))
print(df_col_list)

***
### We want to extract the metadata from the metadata.xls file

The metadata is needed before we start extracting the data from the MB files to create the DataFrame with all the desired columns.


In [ ]:
#The actual name of the excel sheet is "Sheet1" and not "Sheet 1" as in the other files.
metadata_raw = pd.read_excel(MICROBIOME_FOLDER+"/metadata.xls", sheetname='Sheet1', header=0)
metadata_raw.columns = metadata_raw.columns.str.lower()

In [ ]:
## Extract group phase and group type
groups = metadata_raw["group"]
group_type = []
group_phase = []
for group in groups:
    # A special case for the extraction control, we don't want to split it
    if group == "EXTRACTION CONTROL":
        group_type.append(group)
        group_phase.append("")
    else:
        type, phase, *_ = group.split()
        group_type.append(type)
        group_phase.append(phase)

In [ ]:
metadata_raw['group_type'] = pd.Series(group_type)
metadata_raw['group_phase'] = pd.Series(group_phase)
metadata = metadata_raw.drop("group",  axis=1)
metadata

In [ ]:
# Make sure SAMPLE doesn't contain NaN, replace it by an empty string
# This will give use prettier column names later on
metadata["sample"].fillna(value="", inplace=True)
metadata

The format of the metadata is now as we want it

***
### We now want to extract the data from the datasheets

** We create a function to extract the classifications **

We want to extract the classifications from the first column of the datasheets. We will need to do string parsing. Regex are a great tool to remove the unwanted characters from the format. The only unwanted characters are the double quotes ". Also, as said before, we need to manage the case where we have the name "Incertae Sedis" as a Family (4th position)

In [ ]:
import re

def get_classifiers(classifier_string):
    """Replaces " in the classifier_string and 
    splits the string to have an indexable list
    """
    classified_list = re.subn("\"", "", classifier_string)[0].lower().split()
    
    # Special case management
    if len(classified_list) > 6:
        # We join the first extra location with the family
        classified_list[4] = ' '.join([classified_list[4], classified_list[5]])
        del classified_list[5]
        
        # Allow for harbitrary number of words in the genus
        classified_list[5] = ' '.join(classified_list[6:])
        del classified_list[6:]
        
    return classified_list

** We can now attempt to create the desired dataframe **

In [ ]:
# In this section, we test the analysis of a single datasheet
test_data = pd.read_excel(MICROBIOME_FOLDER+"/MID1.xls", sheetname='Sheet 1', header=None)
test_data.columns = ["raw_classification", "value"]
classifier_df = pd.DataFrame(columns=df_col_list)
classifier_array = [get_classifiers(row.raw_classification) for row in test_data.itertuples()]
classifier_series = pd.DataFrame(classifier_array, columns=scientific_classification)
classifier_series["value"] = test_data["value"]
classifier_series

In [ ]:
# We reorder the metadata here, this will order the resulting data
metadata = metadata[["barcode", "group_phase", "group_type", "sample"]]\
              .sort_values(by=[ 'group_phase', 'group_type', 'sample'])
metadata.head(10)

In [ ]:
# Loop over all the datasheets
clean_data = pd.DataFrame(columns=scientific_classification)
for metadata_row in metadata.itertuples():
    raw_data = pd.read_excel(MICROBIOME_FOLDER+"/"+metadata_row.barcode+".xls", sheetname='Sheet 1', header=None)
    # Change column names to something clearer
    raw_data.columns = ["raw_classification", "value"]
    
    # For each datasheet create a local classified set of data
    classifier_array = [get_classifiers(row.raw_classification) for row in raw_data.itertuples()]
    local_classified = pd.DataFrame(classifier_array, columns=scientific_classification)
    
    # Add the columns that are not the classification to the local set of data 
    local_classified[str(metadata_row.barcode)]  = test_data["value"]
    # Add the local data to the clean DataFrame
    clean_data = pd.merge(clean_data, local_classified, how="outer", on=scientific_classification)

clean_data.head(10)

** Lets find all rows that contains only NaN values **

Those rows are not usefull for the analysis and we will drop them.

In [ ]:
# Get a table with True in place of a row where all the MID values are null
table_of_null_row = pd.isnull(clean_data[metadata.barcode]).all(axis=1)

# Get the associated indexes
index_of_null_row = table_of_null_row[table_of_null_row].index[:]

# Lets check that the value are actually null
clean_data.iloc[index_of_null_row]

In [ ]:
clean_data = clean_data.drop(index_of_null_row)

** Now lets replace all the None values by unknow **

In [ ]:
clean_data.fillna(value="unknown", inplace=True)

In [ ]:
clean_data.head(n=5)

We see that some genus are unknown... This seems a bit odd, so we will search if one other of the term is the actual genus

In [ ]:
unknown_genus = clean_data[clean_data.genus == 'unknown']
unknown_genus

From the list above, we can see that we have many cases where we have a string containing "incertae_sedis". Normally, this nomenclature is used when a genus doesn't have clear parents (https://en.wikipedia.org/wiki/Incertae_sedis). Because of this we will attempt to use the first class after the incertae_sedis as the name of the genus and keep the other values as unknown.

In the case of "(some term)_genera_incertae_sedis", it means that the term is the genus and the rest is unknown, so we will do a special parse for those cases.

In cases where we don't have incertae sedis, we will avoid doing manipulations. We don't want to induce errors in the dataset because we managed poorly the nomenclature for the specific cases. Because there's a small amount of thoses cases left after the analysis, it could be asked to an expert in the domain or obtained with more advanced research on the subject to add a case by case filter

In [ ]:
def get_incertae_genus(class_, order):
    """Extracts genus from either class or order if either of them contain incertae_sedis"""
    for target in [class_, order]:
        genus, *rest = target.split('_incertae_sedis')
        if len(rest) > 0:
            return genus
    return 'unknown'

In [ ]:
genuses = [get_incertae_genus(c, o) for *_, c, o in unknown_genus[['class', 'order']].itertuples()]

In [ ]:
# Insert newly retrieved genus names
clean_data.loc[clean_data.genus == 'unknown', 'genus'] = genuses

In [ ]:
# Find the remaining unknown genus, those will not be managed and would be left for later manual management
clean_data[clean_data.genus == 'unknown']

**Replace the incertae sedis by unknown**

In the cases where we have incertae sedis in one of the column, we will replace the value incertae_sedis by unknown. It keeps the actual information because we already setted the associated genus in the previous step. This will allow for a cleaner overall dataset

In [ ]:
def print_all_incertae_sedis(data_frame):
    """ This function extract all the incertae sedis from the class, order and family of a dataframe"""
    print("Found incertae sedis in:")
    for i, c, o, f in clean_data[["class", "order", "family"]].itertuples():
        if "_incertae_sedis" in c or "incertae sedis" in c:
            print("Class '", c, "' at index ", i)
        elif "_incertae_sedis" in o or "incertae sedis" in o:
            print("Order '", o, "' at index ", i)
        elif "_incertae_sedis" in f or "incertae sedis" in f:
            print("Family '", f, "' at index ", i)    

In [ ]:
# Show that there's incertae sedis in the data frame "clean_data"
print_all_incertae_sedis(clean_data)

In [ ]:
# Use the panda library to find all incertae sedis in the data set and replace it by unknown
for column in clean_data.columns:
    clean_data[column].replace(to_replace=".*incertae[_ ]sedis",
                               value="unknown",
                               inplace=True,
                               regex=True)    

In [ ]:
print_all_incertae_sedis(clean_data)

** Now lets manage the index of the DataFrame **

For now, we consider that every column but the value can be considered as a metadata. 

In [ ]:
indexed_data = clean_data.set_index(list(scientific_classification))
indexed_data.head(5)

In [ ]:
indexed_data.index.is_unique

** Now lets give the column to more meaningfull names **

The ordering of the columns is already managed to give a nice output. This is why we reordered the metadata earlier on.

In [ ]:
pretty_data = pd.DataFrame(data=indexed_data.values,
                           index=indexed_data.index,
                           columns=[metadata.group_phase.get_values(),
                                    metadata.group_type.get_values(),
                                    metadata["sample"].get_values()])
pretty_data.head(5)

In [ ]:
#Lets give a name to the columns
pretty_data.columns.names = ["Group Number", "Group Type", "Sample"]
pretty_data.head(10)

In [ ]:
# Lets export the resulting data for easy referencing
pretty_data.to_csv("resulting_data.csv")

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here